# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

I. J. M. Crossfield  ->  I. J. M. Crossfield  |  ['I. J. M. Crossfield']
E.-M. Ahrer  ->  E.-M. Ahrer  |  ['E.-M. Ahrer']
L. Kreidberg  ->  L. Kreidberg  |  ['L. Kreidberg']
G. Chauvin  ->  G. Chauvin  |  ['G. Chauvin']
T. K. Henning  ->  T. K. Henning  |  ['T. K. Henning']
F. Xu  ->  F. Xu  |  ['F. Xu']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']


X. Zhang  ->  X. Zhang  |  ['X. Zhang']
J. Davies  ->  J. Davies  |  ['J. Davies']
Arxiv has 73 new papers today
          5 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/5 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2509.14318


extracting tarball to tmp_2509.14318... done.


/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'ms.bbl' from 'tmp_2509.14318/ms.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure 20250512_s_number_fractions_pies
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:137: LatexWarning: Recovered figure 20250512_s_number_fractions_pies as tmp_2509.14318/./20250512_s_number_fractions_pies.pdf
  warnings.warn(LatexWarning(f'Recovered figure {image} as {fname}'))


...

extracting tarball to tmp_2509.14691...

 done.
Retrieving document from  https://arxiv.org/e-print/2509.15039


extracting tarball to tmp_2509.15039... done.
Retrieving document from  https://arxiv.org/e-print/2509.15131


extracting tarball to tmp_2509.15131...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.14318-b31b1b.svg)](https://arxiv.org/abs/2509.14318) | **Mapping the SO2 Shoreline in Gas Giant Exoplanets**  |
|| <mark>I. J. M. Crossfield</mark>, et al. -- incl., <mark>E.-M. Ahrer</mark>, <mark>L. Kreidberg</mark> |
|*Appeared on*| *2025-09-19*|
|*Comments*| *16 pages, 7 figures, >900 models and spectra at this https URL . Resubmitted to ApJ (on 29 Aug) after referee review*|
|**Abstract**|            JWST has revealed sulfur chemistry in giant exoplanet atmospheres, where molecules such as SO2 trace photochemistry, metallicity, and formation and migration. To ascertain the conditions that determine whether (or how much) SO2, H2S, and other sulfur-bearing species are present in exoplanet atmospheres, we present a grid of planetary atmospheres covering metallicities from 0.3-1000x Solar and temperatures from 250-2050 K. These models map out the 'SO2 shoreline,' the region of metallicity and irradiation for which SO2 may be sufficiently abundant to be detectable. SO2 is a sensitive indicator of metallicity; expected SO2 abundances also depend strongly on overall temperature and C/O ratio; the SO2 abundance depends surprisingly weakly on XUV irradiation, also weakly on Kzz (for Teq > 600 K), and is essentially independent of internal temperature. Despite its detection in a growing number of giant planets, SO2 is never the dominant sulfur-bearing molecule: depending on temperature and metallicity, H2S, S2, NS, SO, SH, and even S8 or atomic S are frequently as common (or more so) as SO2. Nonetheless SO2 remains the most easily detectable sulfur-bearing species, followed by H2S, though perhaps SO and SH could be detectable in some gas giants. Aside from a pressing need for additional observational constraints on sulfur, we also identify the need for future work to account for the effects of clouds and hazes, fully self-consistent atmospheric models, 2D and 3D models, a wider range of planetary masses and radii, and studies to measure and refine reaction rates and molecular opacities of sulfur-bearing species         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.14321-b31b1b.svg)](https://arxiv.org/abs/2509.14321) | **New Orbital Constraints for YSES 1 b and HR 2562 B from High-Precision Astrometry and Planetary Radial Velocities**  |
|| J. Roberts, et al. -- incl., <mark>G. Chauvin</mark>, <mark>T. K. Henning</mark> |
|*Appeared on*| *2025-09-19*|
|*Comments*| **|
|**Abstract**|            We present new VLTI/GRAVITY astrometry and updated orbit fits for the directly imaged companions YSES 1 b and HR 2562 B, substellar objects straddling the planet-brown dwarf boundary. Using high-precision astrometry, radial velocity (RV) data, and proper motions, we derive revised orbital parameters with orbitize! arXiv:1910.01756. For YSES 1 b, the inclusion of GRAVITY astrometry and a relative radial velocity measurement from arXiv:2409.16660 overcomes the traditional challenge of constraining eccentricities for distant companions, enabling the first orbit fit and yielding a constrained eccentricity of 0.44 (0.20). This represents the first full orbit fit for the system. Additionally, we calculate a median line-of-sight stellar obliquity of 12 (+11, -8) degrees, providing further insight into the system's dynamical architecture. For HR 2562 B, our analysis agrees with arXiv:2302.04893, confirming a low-eccentricity orbit (0.34 (0.20)) and an inclination of 87 (1) degrees. We find HR 2562 B's orbit to be nearly coplanar with the debris disk, with a mutual inclination of 3.7 (0.3) degrees. For both YSES 1 b and HR 2562 B the lower eccentricities favor an in situ formation scenario over extreme scattering or cloud fragmentation.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.14691-b31b1b.svg)](https://arxiv.org/abs/2509.14691) | **MCI: Multi-Channel Imager on the Chinese Space Station Survey Telescope**  |
|| Z.-Y. Zheng, et al. -- incl., <mark>F. Xu</mark>, <mark>X. Zhang</mark> |
|*Appeared on*| *2025-09-19*|
|*Comments*| *11 pages, 5 figures, submitted to RAA. Comments are welcome!*|
|**Abstract**|            The Multi-Channel Imager (MCI) is a powerful near-ultraviolet (NUV) and visible imager onboard the Chinese Space Station Survey Telescope (CSST). The MCI provides three imaging channels, which are the NUV channel, the Optical-blue channel and the Optical-red channel, with the wavelength range of 255-430 nm, 430-700 nm, and 700-1000 nm, respectively. The three channels can target the same field simultaneously. Each channel employs a CCD focal plane of 9216 x 9232 pixels and $\sim$7.5 x 7.5 arcmin$^2$ field of view. The MCI's three channels feature unprecedented sensitivities and field of views, as well as rich filter sets, which complements the NUV and visible capabilities of the CSST for the high-precision photometry, the weak-signal detection, and the related sciences. Here we present key design features, results of current ground tests, and suggested observing strategies of the MCI.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.15039-b31b1b.svg)](https://arxiv.org/abs/2509.15039) | **A dark energy model for coasting cosmology**  |
|| Y. Ren, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2025-09-19*|
|*Comments*| *15 pages, 3 figures*|
|**Abstract**|            Coasting cosmology offers an intriguing and straightforward framework for understanding the universe. In this work, we employ the Trans-Planckian Censorship Criterion (TCC) conjecture to test the viability of the coasting cosmology and propose an entropic dark energy (EDE) model within this framework. By applying the holographic principle to constrain the dark energy density and adopting the Bekenstein entropy and Tsallis entropy as the constraining entropies of the system, we find that, in a holographic coasting cosmological framework where dark energy and dark matter evolve independently, the Tsallis entropy satisfies certain general assumptions better than the Bekenstein entropy. Thus, there may be a fundamental relationship between Tsallis entropy and dark energy. We utilize observational data from Type Ia Supernovae (SNIa), Baryon Acoustic Oscillations (BAO), and Cosmic Chronometers (CC) to constrain EDE model. The optimal Tsallis parameter obtained aligns well with theoretical expectations. To evaluate the model's fit to the observed data, we calculate the Akaike Information Criterion (AIC), Bayesian Information Criterion (BIC), and Kullback Information Criterion (KIC), and compare these metrics with those derived from $\Lambda$CDM, under which the model shows some improvement. Overall, this model provides a novel and simple on the evolution of the universe.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2509.15131-b31b1b.svg)](https://arxiv.org/abs/2509.15131) | **Polarimeter to Unify the Corona and Heliosphere (PUNCH)**  |
|| C. DeForest, et al. -- incl., <mark>J. Davies</mark> |
|*Appeared on*| *2025-09-19*|
|*Comments*| *Submitted to the journal Solar Physics; this preprint is not yet peer reviewed*|
|**Abstract**|            The Polarimeter to Unify the Corona and Heliosphere (PUNCH) mission is a NASA Small Explorer to determine the cross-scale processes that unify the solar corona and heliosphere. PUNCH has two science objectives: (1) understand how coronal structures become the ambient solar wind, and (2) understand the dynamic evolution of transient structures, such as coronal mass ejections, in the young solar wind. To address these objectives, PUNCH uses a constellation of four small spacecraft in Sun-synchronous low Earth orbit, to collect linearly polarized images of the K corona and young solar wind. The four spacecraft each carry one visible-light imager in a 1+3 configuration: a single Narrow Field Imager solar coronagraph captures images of the outer corona at all position angles, and at solar elongations from 1.5° (6 R$_\odot$) to 8° (32 R$_\odot$); and three separate Wide Field Imager heliospheric imagers together capture views of the entire inner solar system, at solar elongations from 3° (12 R$_\odot$) to 45° (180 R$_\odot$) from the Sun. PUNCH images include linear-polarization data, to enable inferring the three-dimensional structure of visible features without stereoscopy. The instruments are matched in wavelength passband, support overlapping instantaneous fields of view, and are operated synchronously, to act as a single ``virtual instrument'' with a 90° wide field of view, centered on the Sun. PUNCH launched in March of 2025 and began science operations in June of 2025. PUNCH has an open data policy with no proprietary period, and PUNCH Science Team Meetings are open to all.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2509.14318/./shoreline_plot_v3.png', 'tmp_2509.14318/./shoreline_plot_v3.png', 'tmp_2509.14318/./tps_met_tint.png']
copying  tmp_2509.14318/./shoreline_plot_v3.png to _build/html/
copying  tmp_2509.14318/./shoreline_plot_v3.png to _build/html/
copying  tmp_2509.14318/./tps_met_tint.png to _build/html/
exported in  _build/html/2509.14318.md
    + _build/html/tmp_2509.14318/./shoreline_plot_v3.png
    + _build/html/tmp_2509.14318/./shoreline_plot_v3.png
    + _build/html/tmp_2509.14318/./tps_met_tint.png
found figures ['tmp_2509.14321/./mutual_inclination_histogram.png', 'tmp_2509.14321/./YSESLOS.png', 'tmp_2509.14321/./HR2562_fullcorner_paper.png']
copying  tmp_2509.14321/./mutual_inclination_histogram.png to _build/html/
copying  tmp_2509.14321/./YSESLOS.png to _build/html/
copying  tmp_2509.14321/./HR2562_fullcorner_paper.png to _build/html/
exported in  _build/html/2509.14321.md
    + _build/html/tmp_2509.14321/./mutual_inclination_histogram.png
    + _build/html/

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\ntarg}{26\xspace}$
$\newcommand{\ntoi}{30\xspace}$
$\newcommand{\vsini}{\ensuremath{v \sin i}\xspace}$
$\newcommand{\teff}{\ensuremath{T_\mathrm{eff}}\xspace}$
$\newcommand{\teq}{\ensuremath{T_\mathrm{eq}}\xspace}$
$\newcommand{\tint}{\ensuremath{T_\mathrm{int}}\xspace}$
$\newcommand{\kms}{km~s\ensuremath{^{-1}}\xspace}$
$\newcommand{\kzz}{\ensuremath{K_{zz}}}$
$\newcommand{\soo}{SO_2\xspace}$
$\newcommand{\coo}{CO_2\xspace}$
$\newcommand{\water}{H_2O\xspace}$
$\newcommand{\hhs}{H_2S\xspace}$
$\newcommand{\methane}{CH_4\xspace}$
$\newcommand{\co}[2]{\ensuremath{^{#1}\textrm{C}^{#2}\textrm{O}}\xspace}$
$\newcommand{\car}[1]{\ensuremath{^{#1}\textrm{C}}\xspace}$
$\newcommand{\ox}[1]{\ensuremath{^{#1}\textrm{O}}\xspace}$
$\newcommand{\update}$
$\newcommand{\tirr}{\ensuremath{T_\mathrm{irr}}}$
$\newcommand{\TESS}{{\em TESS}}$
$\newcommand{\tess}{{\em TESS}}$
$\newcommand{\spitzer}{{\em Spitzer}}$
$\newcommand{\gj}{GJ~1252}$
$\newcommand{\gjb}{GJ~1252b}$
$\newcommand{\rp}{1.180}$
$\newcommand{\urp}{\pm0.078}$
$\newcommand{\vesc}{5.4}$
$\newcommand{\uvesc}{\pm0.8}$
$\newcommand{\tday}{1410}$
$\newcommand{\utday}{^{+91}_{-125}}$
$\newcommand{\depth}{149}$
$\newcommand{\udepth}{^{+25}_{-32}}$
$\newcommand{\te}{2458668.3575}$
$\newcommand{\ute}{^{+0.0019}_{-0.0007}}$
$\newcommand{\dt}{+1.4}$
$\newcommand{\udt}{^{+2.8}_{-1.0}}$
$\newcommand{\abUL}{0.41}$
$\newcommand{\fLL}{0.40}$
$\newcommand{\fabLL}{0.37}$
$\newcommand{\ecoswnosign}{0.0025}$
$\newcommand{\ecosw}{+0.0025}$
$\newcommand{\uecosw}{^{+0.0049}_{-0.0018}}$
$\newcommand{\psurfUL}{8.9}$
$\newcommand{\age}{3.9}$
$\newcommand{\uage}{0.4}$
$\newcommand{\referee}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand{\ntarg}{26\xspace}$
$\newcommand{\ntoi}{30\xspace}$
$\newcommand{\vsini}{\ensuremath{v \sin i}\xspace}$
$\newcommand{\teff}{\ensuremath{T_\mathrm{eff}}\xspace}$
$\newcommand{\teq}{\ensuremath{T_\mathrm{eq}}\xspace}$
$\newcommand{\tint}{\ensuremath{T_\mathrm{int}}\xspace}$
$\newcommand{\kms}{km~s\ensuremath{^{-1}}\xspace}$
$\newcommand{\kzz}{\ensuremath{K_{zz}}}$
$\newcommand{\soo}{SO_2\xspace}$
$\newcommand{\coo}{CO_2\xspace}$
$\newcommand{\water}{H_2O\xspace}$
$\newcommand{\hhs}{H_2S\xspace}$
$\newcommand{\methane}{CH_4\xspace}$
$\newcommand{\co}[2]{\ensuremath{^{#1}\textrm{C}^{#2}\textrm{O}}\xspace}$
$\newcommand{\car}[1]{\ensuremath{^{#1}\textrm{C}}\xspace}$
$\newcommand{\ox}[1]{\ensuremath{^{#1}\textrm{O}}\xspace}$
$\newcommand{\update}$
$\newcommand{\tirr}{\ensuremath{T_\mathrm{irr}}}$
$\newcommand{\TESS}{{\em TESS}}$
$\newcommand{\tess}{{\em TESS}}$
$\newcommand{\spitzer}{{\em Spitzer}}$
$\newcommand{\gj}{GJ~1252}$
$\newcommand{\gjb}{GJ~1252b}$
$\newcommand{\rp}{1.180}$
$\newcommand{\urp}{\pm0.078}$
$\newcommand{\vesc}{5.4}$
$\newcommand{\uvesc}{\pm0.8}$
$\newcommand{\tday}{1410}$
$\newcommand{\utday}{^{+91}_{-125}}$
$\newcommand{\depth}{149}$
$\newcommand{\udepth}{^{+25}_{-32}}$
$\newcommand{\te}{2458668.3575}$
$\newcommand{\ute}{^{+0.0019}_{-0.0007}}$
$\newcommand{\dt}{+1.4}$
$\newcommand{\udt}{^{+2.8}_{-1.0}}$
$\newcommand{\abUL}{0.41}$
$\newcommand{\fLL}{0.40}$
$\newcommand{\fabLL}{0.37}$
$\newcommand{\ecoswnosign}{0.0025}$
$\newcommand{\ecosw}{+0.0025}$
$\newcommand{\uecosw}{^{+0.0049}_{-0.0018}}$
$\newcommand{\psurfUL}{8.9}$
$\newcommand{\age}{3.9}$
$\newcommand{\uage}{0.4}$
$\newcommand{\referee}$</div>



<div id="title">

# Mapping the $\soo$ shoreline in gas giant exoplanets

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2509.14318-b31b1b.svg)](https://arxiv.org/abs/2509.14318)<mark>Appeared on: 2025-09-19</mark> -  _16 pages, 7 figures, >900 models and spectra at this https URL . Resubmitted to ApJ (on 29 Aug) after referee review_

</div>
<div id="authors">

<mark>I. J. M. Crossfield</mark>, et al. -- incl., <mark>E.-M. Ahrer</mark>, <mark>L. Kreidberg</mark>

</div>
<div id="abstract">

**Abstract:** JWST has revealed sulfur chemistry in giant exoplanet atmospheres, where molecules such as SO2 trace photochemistry, metallicity, and formation and migration. To ascertain the conditions that determine whether (or how much) $\soo$ , $\hhs$ , and other sulfur-bearing species are present in exoplanet atmospheres, we present a grid of planetary atmospheres covering metallicities from 0.3--1000 $\times$ Solar and temperatures from 250--2050 K. These models map out the ` $\soo$ shoreline,' the region of metallicity and irradiation for which $\soo$ may be sufficiently abundant to be detectable. $\soo$ is a sensitive indicator of metallicity; expected $\soo$ abundances also depend strongly on overall temperature and C/O ratio; the $\soo$ abundance depends surprisingly weakly on XUV irradiation, also weakly on $\kzz$ (for $T_\mathrm{eq} \gtrsim 600$ K), and is essentially independent of internal temperature. Despite its detection in a growing number of giant planets, $\soo$ is never the dominant sulfur-bearing molecule: depending on temperature and metallicity, $H_2$ S, $S_2$ , NS, SO, SH, and even $S_8$ or atomic S are frequently as common (or more so) as $\soo$ . Nonetheless $\soo$ remains the most easily detectable sulfur-bearing species, followed by $\hhs$ , though perhaps SO and SH could be detectable in some gas giants. Aside from a pressing need for additional observational constraints on sulfur, we also identify the need for future work to account for the effects of clouds and hazes, fully self-consistent atmospheric models, 2D and 3D models, a wider range of planetary masses and radii, and studies to measure and refine reaction rates and molecular opacities of sulfur-bearing species.

</div>

<div id="div_fig1">

<img src="tmp_2509.14318/./shoreline_plot_v3.png" alt="Fig3" width="100%"/>

**Figure 3. -** $\soo$ abundance averaged from 1--100 $\mu$bar for our
  nominal model grid (color map), with the 1 ppm volume mixing ratio
  indicated by the dashed line.  We term this the "$\soo$ shoreline,"
  below which $\soo$ is unlikely to be detectable.  Markers show the
  shoreline in the context of exoplanets observed by JWST in transit
  with $R_P\ge 1.8 R_\oplus$.  Empty symbols indicate unpublished
  observations; white hexagons show $\soo$ nondetections while blue
  hexagons show $\soo$ detections. {$\referee$ All four non-detections within the $\soo$-rich region still allow for the presence of significant levels  of $\soo$.} (*fig:shorelinemap*)

</div>
<div id="div_fig2">

<img src="tmp_2509.14318/./shoreline_plot_v3.png" alt="Fig9" width="100%"/>

**Figure 9. -** $\soo$ abundance averaged from 1--100 $\mu$bar for our
  nominal model grid (color map), with the 1 ppm volume mixing ratio
  indicated by the dashed line.  We term this the "$\soo$ shoreline,"
  below which $\soo$ is unlikely to be detectable.  Markers show the
  shoreline in the context of exoplanets observed by JWST in transit
  with $R_P\ge 1.8 R_\oplus$.  Empty symbols indicate unpublished
  observations; white hexagons show $\soo$ nondetections while blue
  hexagons show $\soo$ detections. {$\referee$ All four non-detections within the $\soo$-rich region still allow for the presence of significant levels  of $\soo$.} (*fig:shorelinemap*)

</div>
<div id="div_fig3">

<img src="tmp_2509.14318/./tps_met_tint.png" alt="Fig1" width="100%"/>

**Figure 1. -** Representative temperature profiles of our models, showing
  two metallicities and two $T_\mathrm{int}$ for equilibrium
  temperatures of 250, 700, 1150, 1600, and 2050 K. Changing
  $T_\mathrm{int}$ affects the temperature profile at
  $P\gtrsim1$ mbar, at altitudes below where transmission spectra
  probe and where $\soo$ is produced. (*fig:tps*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2509.14318"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\mstar}{\ensuremath{M_\star}}$
$\newcommand{\msun}{\ensuremath{M_\odot}}$
$\newcommand{\mjup}{\ensuremath{M_J}}$
$\newcommand{\rjup}{\ensuremath{R_J}}$
$\newcommand{\ms}{\ensuremath{\mathrm{km s^{-1}}}}$
$\newcommand{\kms}{\ensuremath{\mathrm{km s^{-1}}}}$
$\newcommand{\red}[1]{\textcolor{red}{#1}}$
$\newcommand{\blue}[1]{\textcolor{blue}{#1}}$
$\newcommand{\highlight}[1]{\textbf{#1}}$</div>



<div id="title">

# New Orbital Constraints for YSES 1 b and HR 2562 B from High-Precision Astrometry and Planetary Radial Velocities$\footnote{Based on observations collected at the European Southern Observatory under ESO programmes 109.238N.002, 110.23U4.001, 109.238N.004, 113.26QU.001, 1104.C-0651(B), 109.238N.002, 109.238N.004, 60.A-9102(J), and 114.27UV.001}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2509.14321-b31b1b.svg)](https://arxiv.org/abs/2509.14321)<mark>Appeared on: 2025-09-19</mark> - 

</div>
<div id="authors">

J. Roberts, et al. -- incl., <mark>G. Chauvin</mark>, <mark>T. K. Henning</mark>

</div>
<div id="abstract">

**Abstract:** We present new VLTI/GRAVITY astrometry and updated orbit fits for the directly imaged companions YSES 1 b and HR 2562 B, substellar objects straddling the planet-brown dwarf boundary. Using high-precision astrometry, radial velocity (RV) data, and proper motions, we derive revised orbital parameters with \texttt{orbitize!} \citep{Blunt_2020} . For YSES 1 b, the inclusion of GRAVITY astrometry and a relative radial velocity measurement from \citet{zhang2024} overcomes the traditional challenge of constraining eccentricities for distant companions, enabling the first orbit fit and yielding a constrained eccentricity of $0.44\pm0.20$ . This represents the first full orbit fit for the system. Additionally, we calculate a median line-of-sight stellar obliquity of $12^{+11}_{-8}$ deg, providing further insight into the system's dynamical architecture. For HR 2562 B, our analysis agrees with \citet{Zhang_2023} , confirming a low-eccentricity orbit ( $0.34\pm0.20$ ) and an inclination of $87\pm 1$ deg. We find HR 2562 B's orbit to be nearly coplanar with the debris disk, with a mutual inclination of $3.7\pm 0.3$ deg. For both YSES 1 b and HR 2562 B the lower eccentricities favor an ${in situ}$ formation scenario over extreme scattering or cloud fragmentation.

</div>

<div id="div_fig1">

<img src="tmp_2509.14321/./mutual_inclination_histogram.png" alt="Fig1" width="100%"/>

**Figure 1. -** Histogram of the mutual inclination between the orbit of HR 2562 B and its disk. The distribution peaks median value of $3.70^\circ$, indicating that HR 2562 B's orbit is nearly coplanar with the disk. (*fig:mutual_inc*)

</div>
<div id="div_fig2">

<img src="tmp_2509.14321/./YSESLOS.png" alt="Fig2" width="100%"/>

**Figure 2. -** Probability density distribution of the Line-of-Sight stellar obliquity for YSES 1. The median obliquity is 11.5° (red dashed line), with the 16th and 84th percentiles at 3.4° and 22.7° (black dotted lines), respectively.
 (*fig:yses_3*)

</div>
<div id="div_fig3">

<img src="tmp_2509.14321/./HR2562_fullcorner_paper.png" alt="Fig3" width="100%"/>

**Figure 3. -** Full Corner plot of HR 2562 B. The estimated posteriors are on the diagonals. The cross-sections represent the covariance with the 1, 2, and 3-$\sigma$ errors.
 (*fig:HR2562_large*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2509.14321"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

133  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

17  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

7  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
